# [IAPR 2019:][iapr2019] Special project

**Group members:**
    1- first name and last name,
    2- first name and last name,
    3- first name and last name

**Due date:** 30.05.2019

[iapr2019]: https://github.com/LTS5/iapr-2019


## Description
Please find the description of this special project via [this link].

[this link]: https://github.com/LTS5/iapr-2019/blob/master/project/special_project_description.pdf

In [49]:
import tarfile
import os
import numpy as np

import skimage.io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

import math
import skimage.color


data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'project-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')

# with tarfile.open(tar_path, mode='r:gz') as tar:
#     tar.extractall(path=data_base_path)

# **** type = test, train, validation ****
type = "test"

# Load images
file_ = open(os.path.join(data_base_path, data_folder, type + ".txt"), "r")
im_names = []
name_i = file_.readline()
im_names.append(name_i[0:(len(name_i)-1)])
while (name_i):
    name_i = file_.readline()
    im_names.append(name_i[0:(len(name_i)-1)])

filenames = [os.path.join(data_base_path, data_folder,"images", type, name) + '.jpg' for name in im_names]
test = skimage.io.imread_collection(filenames)
print('Number of images: ', len(test))


Number of images:  51


## Part 1: Finding varroas by segmentation
Add your implementation for ''**detect_by_segmentation**'' function. Please make sure the input and output follows the mentioned format.

In [40]:
from skimage.measure import label, regionprops
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
from skimage.morphology import disk, square


# Function that rescales the intensity value and sets it between 0 and 255
def rescale_intensity(image):
    y_im, x_im, z_im = image.shape
    out = image.copy()
    for i in range(0,z_im):
        image2 = image[:,:,i]
        minimum = np.min(image2)
        maximum = np.max(image2)
        out[:,:,i]=(image2 - minimum)/(maximum-minimum)*255
    return out

# Function that, using K-Means, finds the centers of each element in the image that is not part of the background.
# Furthermore, each element is labelled
def treshold_varroa(image):
    out = image[:,:,0].copy()
    image = rescale_intensity(image)
    y_im, x_im, z_im = image.shape
    X_r = np.reshape(image[:,:,0], (x_im*y_im))
    X_g = np.reshape(image[:,:,1], (x_im*y_im))
    X_b = np.reshape(image[:,:,2], (x_im*y_im))

    X = np.array([X_r, X_g, X_b])
    X = np.transpose(X)

    #plt.scatter(X[:, 0], X[:, 2], s=1);

    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=3)
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)

    #plt.scatter(X[:, 0], X[:, 2], c=y_kmeans, s=1, cmap='viridis')

    centers = kmeans.cluster_centers_
    #plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

    y_kmeans = np.reshape(y_kmeans, (y_im,x_im))
    get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    label_ = centers[:,0].argmin()
    for x in range(0,x_im):
        for y in range(0,y_im):
            if y_kmeans[y,x]==label_ :
                out[y,x]=0
            else:
                out[y,x]=255
    return out


def inverse_image(image):
    # This function only works for thresholded images and it inverses the blak pixels with the white pixels and vice versa.
    # It is usefull when we use morphological operations because they work on the logical ones and not on the zeros.
    out = image.copy()
    y_image, x_image = image.shape
    for x in range(0,x_image):
        for y in range(0,y_image):
            if image[y,x]==0:
                out[y,x]=255
            else:
                out[y,x]=0
    return out

# Function that creates a histogram which counts the varroas in each image
def create_histogram(image, xmin, ymin, xmax, ymax):
    # This function looks in a rectangle from xmin to xmax, from ymin to ymax how many different labels are present and it
    # create a histogram
    max_label = np.max(image)
    count_label_j = np.zeros(max_label-1)
    for x in range(xmin,xmax+1):
        for y in range(ymin,ymax+1):
            for j in range(0,max_label-1):
                # j+2 because the array starts at zero and the label at 1, and the lable 1 corresponds to the background and we don't want it 
                if image[y,x] == j+2 :
                    count_label_j[j] = count_label_j[j] + 1
    return count_label_j

def extract_varroa(image_lab, label_varroa):
    # This function extracts the varroa having the label "label_varroa" on the image "i"
    # The output is a thresholded image with the varroa's pixels in black, the function also returns the number 
    # of pixels of this varroa
    image = image_lab.copy()
    y_image, x_image = image.shape
    for x in range(0,x_image):
        # This is an inversed threshold, because the morphological operations are working on the true values
        for y in range(0,y_image):
            if image_lab[y,x] == label_varroa:
                image[y,x] = 255
            else:
                image[y,x] = 0
    # We applie a dilation because previously we applied an erosion in order to separate the superposed varroas
    # and now we would like to obtain the entire varroa
    image = skimage.morphology.dilation(image,disk(int(varona_size*0.85)))
    
    # After the morphological operation we reverse the image
    image = inverse_image(image)
    
    
    x_min = x_image
    x_max = 0
    y_min = y_image
    y_max = 0
    for x in range(0,x_image):
        for y in range(0,y_image):
            if image[y,x]==0:
                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
    h = y_max-y_min
    w = x_max-x_min
    
    return x_min, y_min, w, h

little_hole = 2
varona_size = 6

In [43]:
def detect_by_segmentation(img):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''
    # Patrick wrote here
    
    
    image = treshold_varroa(img).copy()
    # We inverse the image because the morphological operations work only on the true elements and not with
    # the false
    image = inverse_image(image)
    
    # We close the little holes present in the varroas
    image = skimage.morphology.closing(image,square(little_hole))
    
    # We open with a circle a bit bigger than the varroa size and we subtract it from the original image
    # with this apprach we eliminate the parts bigger than a varroa
    image = image - skimage.morphology.opening(image,disk(int(varona_size*1.7)))
    
    # We open with a circle a bit smaller than the size of the varroa, like this all the parts smaller than a varroa are deleted
    image = skimage.morphology.opening(image,disk(varona_size))
    
    # In order to label correctly the varroas they don't have to be superposed and we do an erosion with
    # a circle smaller than the size of the varroa
    image = skimage.morphology.erosion(image,disk(int(varona_size*0.85)))
    
    image = inverse_image(image)
    image_thresholded2 = image.copy()
    image_thresholded2 = image.astype(int)*255
    
    image_labeled = skimage.measure.label(image_thresholded2, connectivity=2, background=255)
    
    nbr_varroas = np.max(image) - 1
    
    # for label_varroa from 2 to nbr_varroas+1
    label_varroa = 2
    x_min, y_min, w, h = extract_varroa(image_labeled, label_varroa)
    
    return x_min, y_min, w, h
    #Your code

Add your implementation. Report the Precision, Recall and F1-score, by using all 50 images of the test-set, and considering 0.3 as the IoU threshold.

In [48]:


detect_by_segmentation(test[1])

(877, 1297, 15, 16)

## Part 2: Implement your first detector

Write your function(s) for the second part. Feel free to change the name of the function and add your additional functions, but please make sure their input and output follows the mentioned format.

In [3]:
def detect_by_method_1(img):
    '''
    Input: One single image
    Output: A numpy array containing coordonates of all detected varroas, with the following format: 
            [[x_1, y_1, w_1, h_2], [x_2, y_2, w_1, h_2], ..., [x_n, y_n, w_n, h_n]] 
            where ''n'' is the number of detected varroas.
    '''

    #Your code

Add your implementation. Report the Precision, Recall and F1-score, by using all 50 images of the test-set, and considering 0.3 as the IoU threshold.

In [4]:
#Your code

## Part 3: Using MLP and CNNs

Add your implementation for the thrid part. Feel free to add your desirable functions, but please make sure you have proper functions for the final detection, where their input and output follows the same format as the previous parts.

In [5]:
#Your code

## Challenge

You can generate a json submission file by using the function ''**generate_pred_json**''. This prediction file can be uploaded online for evaluation (Please refer to section 3 of the project description for more details).

In [6]:
import numpy as np
import json

def generate_pred_json(data, tag='baseline'):
    '''
    Input
    - data: Is a dictionary d, such that:
          d = { 
              "ID_1": [], 
              "ID_2": [[x_21, y_21, w_21, h_21], [x_22, y_22, w_22, h_22]], 
              ... 
              "ID_i": [[x_i1, y_i1, w_i1, h_i1], ..., [x_iJ, y_iJ, w_iJ, h_iJ]],
              ... 
              "ID_N": [[x_N1, y_N1, w_N1, h_N1]],
          }
          where ID is the string id of the image (e.i. 5a05e86fa07d56baef59b1cb_32.00px_1) and the value the Kx4 
          array of intergers for the K predicted bounding boxes (e.g. [[170, 120, 15, 15]])
    - tag: (optional) string that will be added to the name of the json file.
    Output
      Create a json file, "prediction_[tag].json", conatining the prediction to EvalAI format.
    '''
    unvalid_key = []
    _data = data.copy()
    for key, value in _data.items():
        try:
            # Try to convert to numpy array and cast as closest int
            print(key)
            v = np.around(np.array(value)).astype(int)
            # Check is it is a 2d array with 4 columns (x,y,w,h)
            if v.ndim != 2 or v.shape[1] != 4:
                unvalid_key.append(key)
            # Id must be a string
            if not isinstance(key, str):
                unvalid_key.append(key)
            _data[key] = v.tolist()
        # Deal with not consistant array size and empty predictions
        except (ValueError, TypeError):
            unvalid_key.append(key)
    # Remove unvalid key from dictionnary
    for key in unvalid_key: del _data[key]
    
    with open('prediction_{}.json'.format(tag), 'w') as outfile:
        json.dump(_data, outfile)

In [7]:
#Your code